In [2]:
# initialize 
from pybaseball import statcast, cache, playerid_lookup, statcast_pitcher, statcast_batter
from datetime import datetime
import pandas as pd
cache.enable()
start_dt = '2024-03-28'
end_dt = datetime.now().date().strftime('%Y-%m-%d')

In [3]:
batters = [
    "Logan O'Hoppe",
    "Justin Turner",
    "Ketel Marte",
    "Max Muncy",
    "Ezequiel Tovar",
    "Juan Soto",
    "Steven Kwan",
    "Michael Conforto",
    "Aaron Judge",
    "Tyler O'Neill",
    # "Luis Robert Jr" # gotta figure out the suffix
]
pitchers = [
    "Yoshinobu Yamamoto",
    "Josh Hader",
    "Sonny Gray",
    "Kenley Jansen",
    "Cristian Javier",
    "Hunter Greene",
    "Carlos Rodon",
    "Aaron Nola",
    "Ronel Blanco",
    "Nick Pivetta",
]

batter_ids = [
    playerid_lookup(split_name[1], split_name[0])
    for split_name in [name.split(" ") for name in batters]
]
pitcher_ids = [
    playerid_lookup(split_name[1], split_name[0])
    for split_name in [name.split(" ") for name in pitchers]
]
roster_ids = batter_ids + pitcher_ids

In [4]:
def getRosterStats(player_ids, statcast_func, start_dt=start_dt, end_dt=end_dt):
    key_mlbams = [player_id['key_mlbam'] for player_id in player_ids if 'key_mlbam' in player_id]
    player_stats = [statcast_func(start_dt, end_dt, player_id=key_mlbam[0]) for key_mlbam in key_mlbams if len(key_mlbam) > 0]
    return pd.concat(player_stats)

In [5]:
batter_stats = getRosterStats(batter_ids, statcast_batter)
pitcher_stats = getRosterStats(pitcher_ids, statcast_pitcher)

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


In [6]:
batter_stats.to_csv('exports/batter_stats.csv',index=False)
pitcher_stats.to_csv('exports/pitcher_stats.csv',index=False)

In [15]:
average_velocity = pitcher_stats[['player_name','release_speed','pitch_name']] \
  .groupby(['player_name','pitch_name']) \
  .agg(['mean']) \
  .sort_values(by=('release_speed','mean'),ascending=False).reset_index().set_index('player_name')
  # .sort_index(level='player_name') 
average_velocity.style.set_caption("Average Pitch Velocity by Pitcher").format(precision=2)

,pitch_name,release_speed
,,mean
player_name,,
"Greene, Hunter",4-Seam Fastball,98.52
"Hader, Josh",Sinker,95.64
"Yamamoto, Yoshinobu",4-Seam Fastball,95.34
"Jansen, Kenley",Sinker,94.57
"Pivetta, Nick",4-Seam Fastball,94.09
"Blanco, Ronel",4-Seam Fastball,93.50
"Gray, Sonny",Sinker,93.09
"Gray, Sonny",4-Seam Fastball,92.70


In [8]:
average_velocity.to_html('exports/average_velocity.html')

In [9]:
start_dt = '2024-03-28'
end_dt = datetime.now().date().strftime('%Y-%m-%d')
statcast(start_dt, end_dt, ).to_csv(f'exports/statcast_{start_dt}-{end_dt}.csv', index=False)

This is a large query, it may take a moment to complete


100%|██████████| 20/20 [00:01<00:00, 19.85it/s]
c:\Users\kylebrague\Code\Sandboxes\baseball\.venv\Lib\site-packages\pybaseball\statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)
